In [1]:
pip install pandas numpy pulp streamlit


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.1.1 -> 24.1.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
import pandas as pd

# Load datasets
order_list = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - OrderList.csv")

In [3]:
import pandas as pd

# Load datasets
order_list = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - OrderList.csv")
freight_rates = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - FreightRates.csv")
plant_ports = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - PlantPorts.csv")
products_per_plant = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - ProductsPerPlant.csv")
vmi_customers = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - VmiCustomers.csv")
wh_capacities = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - WhCapacities.csv")
wh_costs = pd.read_csv("C:\\Users\\Vignesh T\\Downloads\\Supply chain logisitcs problem.xlsx - WhCosts.csv")

# Data cleansing function
def clean_data(df):
    df.dropna(inplace=True)
    df.columns = df.columns.str.strip()
    return df

# Clean all datasets
order_list = clean_data(order_list)
freight_rates = clean_data(freight_rates)
plant_ports = clean_data(plant_ports)
products_per_plant = clean_data(products_per_plant)
vmi_customers = clean_data(vmi_customers)
wh_capacities = clean_data(wh_capacities)
wh_costs = clean_data(wh_costs)

# Save cleaned data for further use
order_list.to_csv('cleaned_OrderList.csv', index=False)
freight_rates.to_csv('cleaned_FreightRates.csv', index=False)
plant_ports.to_csv('cleaned_PlantPorts.csv', index=False)
products_per_plant.to_csv('cleaned_ProductsPerPlant.csv', index=False)
vmi_customers.to_csv('cleaned_VmiCustomers.csv', index=False)
wh_capacities.to_csv('cleaned_WhCapacities.csv', index=False)
wh_costs.to_csv('cleaned_WhCosts.csv', index=False)


In [6]:
import pandas as pd

# Assuming order_list and wh_costs are DataFrames and imported pandas as pd

# Debugging: Print columns of both DataFrames to ensure 'Warehouse ID' and 'Product ID' exist
print("Columns in order_list:", order_list.columns)
print("Columns in wh_costs:", wh_costs.columns)

# Step 1: Merge order_list with wh_costs
try:
    order_list = pd.merge(order_list, wh_costs, on=['Warehouse ID', 'Product ID'], how='left')
    print("Merge successful. Columns in merged DataFrame:", order_list.columns)
except KeyError as e:
    print("KeyError during merge:", e)

# Step 2: Calculate Historical Cost
try:
    order_list['Historical Storage Cost'] = order_list['Unit Quantity'] * order_list['Cost per Unit']
    order_list['Total Historical Cost'] = order_list['Historical Cost'] + order_list['Historical Storage Cost']
    print("Calculation of Historical Costs successful.")
except KeyError as e:
    print("KeyError during calculation:", e)

# Step 3: Save result
try:
    order_list.to_csv('HistoricalCosts.csv', index=False)
    print("CSV file saved successfully.")
except Exception as e:
    print("Error saving CSV file:", e)


Columns in order_list: Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')
Columns in wh_costs: Index(['WH', 'Cost/unit'], dtype='object')
KeyError during merge: 'Warehouse ID'
KeyError during calculation: 'Unit Quantity'
CSV file saved successfully.


In [10]:
import pandas as pd

# Assuming order_list and wh_capacities are DataFrames and imported pandas as pd

# Debugging: Print columns of order_list
print("Columns in order_list:", order_list.columns)

# Convert 'Order Date' to datetime
try:
    order_list['Order Date'] = pd.to_datetime(order_list['Order Date'])
    print("Order Date conversion successful.")
except KeyError as e:
    print("KeyError during Order Date conversion:", e)
    order_list['Order Date'] = None  # Ensure the column is set to None in case of error

# Ensure 'Order Date' conversion was successful
if order_list['Order Date'] is not None:
    # Group by 'Warehouse ID' and date to calculate daily orders
    try:
        daily_orders = order_list.groupby(['Warehouse ID', order_list['Order Date'].dt.date]).size().reset_index(name='Daily Orders')
        print("Daily orders calculation successful. Columns in daily_orders:", daily_orders.columns)
    except KeyError as e:
        print("KeyError during groupby operation:", e)
        daily_orders = None  # Ensure the variable is set to None in case of error
    except NameError as e:
        print("NameError during groupby operation:", e)
        daily_orders = None  # Ensure the variable is set to None in case of error

    # Ensure daily_orders was created successfully
    if daily_orders is not None:
        # Debugging: Print columns of wh_capacities
        print("Columns in wh_capacities:", wh_capacities.columns)

        # Merge daily_orders with wh_capacities
        try:
            capacity_utilization = pd.merge(daily_orders, wh_capacities, on='Warehouse ID')
            print("Merge successful. Columns in capacity_utilization:", capacity_utilization.columns)
        except KeyError as e:
            print("KeyError during merge operation:", e)
            capacity_utilization = None  # Ensure the variable is set to None in case of error
        except NameError as e:
            print("NameError during merge operation:", e)
            capacity_utilization = None  # Ensure the variable is set to None in case of error

        # Ensure capacity_utilization was created successfully
        if capacity_utilization is not None:
            # Calculate capacity utilization
            try:
                capacity_utilization['Utilization'] = capacity_utilization['Daily Orders'] / capacity_utilization['Capacity (Orders per day)'] * 100
                print("Calculation of capacity utilization successful.")
            except KeyError as e:
                print("KeyError during calculation:", e)

            # Save result
            try:
                capacity_utilization.to_csv('CapacityUtilization.csv', index=False)
                print("CSV file saved successfully.")
            except Exception as e:
                print("Error saving CSV file:", e)
        else:
            print("capacity_utilization DataFrame was not created.")
    else:
        print("daily_orders DataFrame was not created.")
else:
    print("Order Date conversion failed.")


Columns in order_list: Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')
Order Date conversion successful.
KeyError during groupby operation: 'Warehouse ID'
daily_orders DataFrame was not created.


In [12]:
import pandas as pd

# Assuming order_list and freight_rates are DataFrames and imported pandas as pd

# Debugging: Print columns of order_list and freight_rates
print("Columns in order_list:", order_list.columns)
print("Columns in freight_rates:", freight_rates.columns)

# Check for column names and print first few rows for better understanding
print("First few rows in order_list:\n", order_list.head())
print("First few rows in freight_rates:\n", freight_rates.head())

# Ensure the necessary columns exist in both DataFrames before merging
if 'Historical Route' in order_list.columns and 'Unit Weight' in order_list.columns and 'Destination' in freight_rates.columns and 'Weight Gap' in freight_rates.columns:
    # Step 1: Merge order_list with freight_rates
    try:
        order_list = pd.merge(order_list, freight_rates, left_on=['Historical Route', 'Unit Weight'], right_on=['Destination', 'Weight Gap'], how='left')
        print("Merge successful. Columns in merged DataFrame:", order_list.columns)
    except KeyError as e:
        print("KeyError during merge operation:", e)
else:
    print("One or more columns for merging are missing in order_list or freight_rates DataFrame.")

# Ensure merge was successful before proceeding
if 'Rate' in order_list.columns:
    # Step 2: Calculate Freight Cost
    try:
        order_list['Freight Cost'] = order_list['Unit Weight'] * order_list['Rate']
        print("Calculation of Freight Cost successful.")
    except KeyError as e:
        print("KeyError during Freight Cost calculation:", e)
else:
    print("'Rate' column is missing in order_list DataFrame after merge.")

# Save result
try:
    order_list.to_csv('FreightCosts.csv', index=False)
    print("CSV file saved successfully.")
except Exception as e:
    print("Error saving CSV file:", e)


Columns in order_list: Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')
Columns in freight_rates: Index(['Carrier', 'orig_port_cd', 'dest_port_cd', 'minm_wgh_qty',
       'max_wgh_qty', 'svc_cd', 'minimum cost', 'rate', 'mode_dsc',
       'tpt_day_cnt', 'Carrier type'],
      dtype='object')
First few rows in order_list:
      Order ID Order Date Origin Port Carrier  TPT Service Level  \
0  1447296447 2013-05-26      PORT09   V44_3    1           CRF   
1  1447158015 2013-05-26      PORT09   V44_3    1           CRF   
2  1447138899 2013-05-26      PORT09   V44_3    1           CRF   
3  1447363528 2013-05-26      PORT09   V44_3    1           CRF   
4  1447363981 2013-05-26      PORT09   V44_3    1           CRF   

   Ship ahead day count  Ship Late Day count   Customer  Prod

In [17]:
import pandas as pd

# Assuming the DataFrames are already defined
print("Columns in order_list:", order_list.columns)
print("Columns in wh_capacities:", wh_capacities.columns)
print("Columns in vmi_customers:", vmi_customers.columns)

# Print a sample of the named tuples
print("Sample order_list named tuple:", next(order_list.itertuples(index=False)))
print("Sample wh_capacities named tuple:", next(wh_capacities.itertuples(index=False)))
print("Sample vmi_customers named tuple:", next(vmi_customers.itertuples(index=False)))

from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, LpVariable, lpSum
import pandas as pd

# Define the problem
model = LpProblem(name="supply-chain-optimization", sense=LpMinimize)

# Define variables
# Replace 'Warehouse_ID' and 'Customer_ID' with actual column names from your DataFrame
variables = {(order['Warehouse ID'], order['Customer ID']): LpVariable(name=f"x_{order['Warehouse ID']}_{order['Customer ID']}", lowBound=0, cat='Continuous') 
             for order in order_list.itertuples(index=False)}

# Define the objective function
# Replace 'Historical_Storage_Cost' and 'Freight_Cost' with actual column names
model += lpSum(variables[order['Warehouse ID'], order['Customer ID']] * (order['Historical Storage Cost'] + order['Freight Cost']) 
               for order in order_list.itertuples(index=False))

# Define constraints
# Replace 'Warehouse_ID' and 'Capacity_Orders_per_day' with actual column names
for warehouse in wh_capacities.itertuples(index=False):
    model += lpSum(variables[warehouse['Warehouse ID'], customer['Customer ID']] 
                   for customer in vmi_customers.itertuples(index=False) 
                   if customer['Warehouse ID'] == warehouse['Warehouse ID']) <= warehouse['Capacity (Orders per day)'], f"Capacity_Constraint_{warehouse['Warehouse ID']}"

# Solve the model
status = model.solve()

# Print the results
print(f"Status: {LpStatus[model.status]}")
for var in model.variables():
    print(f"{var.name}: {var.varValue}")

# Save results
results = pd.DataFrame([(v.name, v.varValue) for v in model.variables()], columns=['Variable', 'Value'])
results.to_csv('OptimizationResults.csv', index=False)


Columns in order_list: Index(['Order ID', 'Order Date', 'Origin Port', 'Carrier', 'TPT',
       'Service Level', 'Ship ahead day count', 'Ship Late Day count',
       'Customer', 'Product ID', 'Plant Code', 'Destination Port',
       'Unit quantity', 'Weight'],
      dtype='object')
Columns in wh_capacities: Index(['Plant ID', 'Daily Capacity'], dtype='object')
Columns in vmi_customers: Index(['Plant Code', 'Customers'], dtype='object')
Sample order_list named tuple: Pandas(_0=1447296447, _1=Timestamp('2013-05-26 00:00:00'), _2='PORT09', Carrier='V44_3', TPT=1, _5='CRF', _6=3, _7=0, Customer='V55555_53', _9=1700106, _10='PLANT16', _11='PORT09', _12=808, Weight=14.3)
Sample wh_capacities named tuple: Pandas(_0='PLANT15', _1=11)
Sample vmi_customers named tuple: Pandas(_0='PLANT02', Customers='V5555555555555_16')


TypeError: tuple indices must be integers or slices, not str

In [19]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, LpVariable, lpSum
import pandas as pd

# Define the problem
model = LpProblem(name="supply-chain-optimization", sense=LpMinimize)

# Create variables dictionary directly from DataFrame
variables = {}
for _, order in order_list.iterrows():
    variables[(order['Plant Code'], order['Customer'])] = LpVariable(
        name=f"x_{order['Plant Code']}_{order['Customer']}", lowBound=0, cat='Continuous'
    )

# Define the objective function
model += lpSum(
    variables[(order['Plant Code'], order['Customer'])] * (order['Unit quantity'] * order['Weight'])
    for _, order in order_list.iterrows()
)

# Define constraints
for _, warehouse in wh_capacities.iterrows():
    model += lpSum(
        variables.get((warehouse['Plant ID'], customer['Customers']), 0)
        for _, customer in vmi_customers.iterrows()
        if customer['Plant Code'] == warehouse['Plant ID']
    ) <= warehouse['Daily Capacity'], f"Capacity_Constraint_{warehouse['Plant ID']}"

# Solve the model
status = model.solve()

# Print the results
print(f"Status: {LpStatus[model.status]}")
for var in model.variables():
    print(f"{var.name}: {var.varValue}")

# Save results
results = pd.DataFrame([(v.name, v.varValue) for v in model.variables()], columns=['Variable', 'Value'])
results.to_csv('OptimizationResults.csv', index=False)


Status: Optimal
x_PLANT03_V555555555555555555_17: 0.0
x_PLANT03_V555555555555555555_42: 0.0
x_PLANT03_V555555555555555555_45: 0.0
x_PLANT03_V555555555555555555_46: 0.0
x_PLANT03_V555555555555555_23: 0.0
x_PLANT03_V555555555555555_29: 0.0
x_PLANT03_V555555555555555_44: 0.0
x_PLANT03_V55555555555555_8: 0.0
x_PLANT03_V5555555555555_16: 0.0
x_PLANT03_V555555555555_31: 0.0
x_PLANT03_V55555555555_28: 0.0
x_PLANT03_V555555555_14: 0.0
x_PLANT03_V555555555_27: 0.0
x_PLANT03_V555555555_3: 0.0
x_PLANT03_V555555555_35: 0.0
x_PLANT03_V55555555_0: 0.0
x_PLANT03_V55555555_32: 0.0
x_PLANT03_V55555555_5: 0.0
x_PLANT03_V55555555_7: 0.0
x_PLANT03_V5555555_12: 0.0
x_PLANT03_V5555555_19: 0.0
x_PLANT03_V5555555_22: 0.0
x_PLANT03_V5555555_30: 0.0
x_PLANT03_V555555_24: 0.0
x_PLANT03_V555555_34: 0.0
x_PLANT03_V555555_40: 0.0
x_PLANT03_V555555_6: 0.0
x_PLANT03_V55555_2: 0.0
x_PLANT03_V55555_26: 0.0
x_PLANT03_V55555_4: 0.0
x_PLANT03_V5555_20: 0.0
x_PLANT03_V5555_25: 0.0
x_PLANT03_V5555_33: 0.0
x_PLANT03_V5555_36

In [20]:
# Example: Scenario with changed warehouse capacities
new_wh_capacities = wh_capacities.copy()
new_wh_capacities['Capacity (Orders per day)'] *= 1.1  # Increase capacities by 10%

# Repeat optimization with new capacities
model = LpProblem(name="supply-chain-optimization", sense=LpMinimize)
variables = {(order['Warehouse ID'], order['Customer ID']): LpVariable(name=f"x_{order['Warehouse ID']}_{order['Customer ID']}", lowBound=0, cat='Continuous') for order in order_list.itertuples()}

model += lpSum(variables[order['Warehouse ID'], order['Customer ID']] * (order['Historical Storage Cost'] + order['Freight Cost']) for order in order_list.itertuples())

for warehouse in new_wh_capacities.itertuples():
    model += lpSum(variables[warehouse['Warehouse ID'], customer['Customer ID']] for customer in vmi_customers.itertuples() if customer['Warehouse ID'] == warehouse['Warehouse ID']) <= warehouse['Capacity (Orders per day)'], f"New_Capacity_Constraint_{warehouse['Warehouse ID']}"

status = model.solve()

print(f"Status: {LpStatus[model.status]}")
for var in model.variables():
    print(f"{var.name}: {var.value()}")

results = pd.DataFrame([(v.name, v.varValue) for v in model.variables()], columns=['Variable', 'Value'])
results.to_csv('ScenarioOptimizationResults.csv', index=False)


KeyError: 'Capacity (Orders per day)'

In [22]:
from pulp import LpMaximize, LpMinimize, LpProblem, LpStatus, LpVariable, lpSum
import pandas as pd

# Create a copy with increased capacities
new_wh_capacities = wh_capacities.copy()
new_wh_capacities['Daily Capacity'] *= 1.1  # Increase capacities by 10%

# Define the problem
model = LpProblem(name="supply-chain-optimization", sense=LpMinimize)

# Define variables
variables = {
    (order['Plant Code'], order['Customer']): LpVariable(
        name=f"x_{order['Plant Code']}_{order['Customer']}", lowBound=0, cat='Continuous'
    )
    for _, order in order_list.iterrows()
}

# Define the objective function
model += lpSum(
    variables[(order['Plant Code'], order['Customer'])] * (order['Unit quantity'] * order['Weight'])
    for _, order in order_list.iterrows()
)

# Define constraints with new capacities
for _, warehouse in new_wh_capacities.iterrows():
    model += lpSum(
        variables.get((warehouse['Plant ID'], customer['Customers']), 0)
        for _, customer in vmi_customers.iterrows()
        if customer['Plant Code'] == warehouse['Plant ID']
    ) <= warehouse['Daily Capacity'], f"New_Capacity_Constraint_{warehouse['Plant ID']}"

# Solve the model
status = model.solve()

# Print the results
print(f"Status: {LpStatus[model.status]}")
for var in model.variables():
    print(f"{var.name}: {var.varValue}")

# Save results
results = pd.DataFrame([(v.name, v.varValue) for v in model.variables()], columns=['Variable', 'Value'])
results.to_csv('ScenarioOptimizationResults.csv', index=False)


Status: Optimal
x_PLANT03_V555555555555555555_17: 0.0
x_PLANT03_V555555555555555555_42: 0.0
x_PLANT03_V555555555555555555_45: 0.0
x_PLANT03_V555555555555555555_46: 0.0
x_PLANT03_V555555555555555_23: 0.0
x_PLANT03_V555555555555555_29: 0.0
x_PLANT03_V555555555555555_44: 0.0
x_PLANT03_V55555555555555_8: 0.0
x_PLANT03_V5555555555555_16: 0.0
x_PLANT03_V555555555555_31: 0.0
x_PLANT03_V55555555555_28: 0.0
x_PLANT03_V555555555_14: 0.0
x_PLANT03_V555555555_27: 0.0
x_PLANT03_V555555555_3: 0.0
x_PLANT03_V555555555_35: 0.0
x_PLANT03_V55555555_0: 0.0
x_PLANT03_V55555555_32: 0.0
x_PLANT03_V55555555_5: 0.0
x_PLANT03_V55555555_7: 0.0
x_PLANT03_V5555555_12: 0.0
x_PLANT03_V5555555_19: 0.0
x_PLANT03_V5555555_22: 0.0
x_PLANT03_V5555555_30: 0.0
x_PLANT03_V555555_24: 0.0
x_PLANT03_V555555_34: 0.0
x_PLANT03_V555555_40: 0.0
x_PLANT03_V555555_6: 0.0
x_PLANT03_V55555_2: 0.0
x_PLANT03_V55555_26: 0.0
x_PLANT03_V55555_4: 0.0
x_PLANT03_V5555_20: 0.0
x_PLANT03_V5555_25: 0.0
x_PLANT03_V5555_33: 0.0
x_PLANT03_V5555_36

In [21]:
# Verify column names in new_wh_capacities
print("Columns in new_wh_capacities:", new_wh_capacities.columns)


Columns in new_wh_capacities: Index(['Plant ID', 'Daily Capacity'], dtype='object')


In [26]:
import pandas as pd

# Assuming you have these DataFrames: order_list, wh_capacities, and vmi_customers

# Calculate Capacity Utilization
order_list['Order Date'] = pd.to_datetime(order_list['Order Date'])
daily_orders = order_list.groupby(['Plant Code', order_list['Order Date'].dt.date]).size().reset_index(name='Daily Orders')
capacity_utilization = pd.merge(daily_orders, wh_capacities, left_on='Plant Code', right_on='Plant ID')

capacity_utilization['Utilization'] = capacity_utilization['Daily Orders'] / capacity_utilization['Daily Capacity'] * 100

# Save result
capacity_utilization.to_csv('CapacityUtilization.csv', index=False)


In [28]:
import streamlit as st
import pandas as pd

# Load datasets
historical_costs = pd.read_csv('HistoricalCosts.csv')
capacity_utilization = pd.read_csv('CapacityUtilization.csv')
freight_costs = pd.read_csv('FreightCosts.csv')
optimization_results = pd.read_csv('OptimizationResults.csv')

# Title
st.title('Supply Chain Optimization Dashboard')

# Historical Costs
st.header('Historical Costs')
st.dataframe(historical_costs)

# Capacity Utilization
st.header('Capacity Utilization')
st.dataframe(capacity_utilization)

# Freight Costs
st.header('Freight Costs')
st.dataframe(freight_costs)

# Optimization Results
st.header('Optimization Results')
st.dataframe(optimization_results)




DeltaGenerator()

In [29]:
import streamlit as st
import pandas as pd
import os

# Define file paths
files = {
    'Historical Costs': 'HistoricalCosts.csv',
    'Capacity Utilization': 'CapacityUtilization.csv',
    'Freight Costs': 'FreightCosts.csv',
    'Optimization Results': 'OptimizationResults.csv'
}

# Function to load data with error handling
def load_data(file_path):
    if os.path.exists(file_path):
        return pd.read_csv(file_path)
    else:
        st.error(f"File not found: {file_path}")
        return pd.DataFrame()  # Return an empty DataFrame if file is not found

# Load datasets
historical_costs = load_data(files['Historical Costs'])
capacity_utilization = load_data(files['Capacity Utilization'])
freight_costs = load_data(files['Freight Costs'])
optimization_results = load_data(files['Optimization Results'])

# Title
st.title('Supply Chain Optimization Dashboard')

# Historical Costs
st.header('Historical Costs')
if not historical_costs.empty:
    st.dataframe(historical_costs)
else:
    st.write("No data available for Historical Costs.")

# Capacity Utilization
st.header('Capacity Utilization')
if not capacity_utilization.empty:
    st.dataframe(capacity_utilization)
else:
    st.write("No data available for Capacity Utilization.")

# Freight Costs
st.header('Freight Costs')
if not freight_costs.empty:
    st.dataframe(freight_costs)
else:
    st.write("No data available for Freight Costs.")

# Optimization Results
st.header('Optimization Results')
if not optimization_results.empty:
    st.dataframe(optimization_results)
else:
    st.write("No data available for Optimization Results.")


In [24]:
import os

# List files in the current directory to check if 'CapacityUtilization.csv' exists
print(os.listdir('.'))


['Cleaned_data.ipynb', 'cleaned_FreightRates.csv', 'cleaned_merged_data.csv', 'cleaned_OrderList.csv', 'cleaned_PlantPorts.csv', 'cleaned_ProductsPerPlant.csv', 'cleaned_VmiCustomers.csv', 'cleaned_WhCapacities.csv', 'cleaned_WhCosts.csv', 'FreightCosts.csv', 'HistoricalCosts.csv', 'HR_Attrition.ipynb', 'Motgage.ipynb', 'OptimizationResults.csv', 'ScenarioOptimizationResults.csv', 'SupplyChain.ipynb', 'Task.ipynb', 'Technocolab_Attrition.ipynb', 'Variate_analysis.ipynb', 'WA_Fn-UseC_-HR-Employee-Attrition.csv']


In [ ]:
# Supply Chain Optimization

## Project Overview
This project aims to develop a Tableau-based analysis model and a Python-based optimization model to optimize the routing of orders from warehouses to customers, ensuring minimal costs while satisfying constraints.

## Technologies
- Python (Pandas, NumPy, PuLP)
- Tableau/Streamlit
- Linear Programming

## Dataset Description
- **OrderList**: Historical order data.
- **FreightRates**: Shipping costs and times.
- **PlantPorts**: Warehouse and port links.
- **ProductsPerPlant**: Warehouse-product relationships.
- **VmiCustomers**: Customer-specific warehouse restrictions.
- **WhCapacities**: Warehouse maximum order capacities.
- **WhCosts**: Storage costs.

## Analysis Tasks
- Historical Cost Calculation
- Capacity Utilization Analysis
- Freight Cost Analysis
- Optimization Model
- Scenario Analysis
- Visualization

## How to Run
1. Clone the repository.
2. Install required packages: `pip install pandas numpy pulp streamlit`
3. Run the data processing scripts to clean and analyze data.
4. Run the optimization model script.
5. Launch the Streamlit app: `streamlit run app.py`

## Deliverables
- **Visualization Dashboard**: Interactive dashboards for key metrics.
- **Summary Report**: Detailed analysis and recommendations.

## Evaluation Metrics
- **Maintainability**: Code structured in modular functions following PEP-8 standards.
- **Portability**: Ensuring compatibility across different environments.
- **GitHub Repository**: Public repository with a proper README detailing project workflow, execution, and setup instructions.
